# Tensor Flow 기본 사용법

In [5]:
import tensorflow as tf

## 1. 개요

Tensor flow는 graph로 연산을 나타내는 프로그래밍 시스템

### 용어 정리
- operation : graph에 있는 노드 = op
- Tensor : 다차원의 배열(python : ndarray)
- graph : 노드(op)와 엣지(data)로 이루어진 구조
- session : graph을 디바이스에 배정하는 메서드 제공

## 2. 동작원리

1. graph 설계 (구성 단계)
2. Session에서 실행 (실행 단계)

In [2]:
# graph 생성

# constant 변수 정의
matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2.], [2.]])

# 두 mat 곱
product = tf.matmul(matrix1, matrix2)

In [4]:
# session 실행
sess = tf.Session()

# 입력한 값 실행 op3개 실행
result = sess.run(product)
print(result)

sess.close()

[[ 12.]]


In [6]:
# with문으로 간단하게
with tf.Session() as sess:
    result = sess.run(product)
    print(result)

[[ 12.]]


## 3. 분산된 session에서 graph 실행

### 3-1. GPU 선택
- 기본적으로 첫번째 GPU이용
- 복수의 GPU 이용시 명시적으로 지정

In [13]:
# 복수 GPU 이용
with tf.Session() as sess:
    with tf.device("/gpu:0"):
        result = sess.run(product)

- `with tf.device()` 이용
- `/cpu:0`, `/gpu:0`, `/gpu:1`

### 3-2. Cluster에서 session 실행
- 클러스터 : 여러 대의 컴퓨터를 연결해 하나의 시스템처럼 동작하는 컴퓨터 집합

In [3]:
# with tf.Session("http://example.org") as sess: # master
#     with tf.device("/job:ps/task:0"): # 특정머신에 작업 분배
#         sess.run()

## 4. Interacticve 사용

In [7]:
sess = tf.InteractiveSession()

x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

# 초기화 op로 x 초기화
x.initializer.run()

sub = tf.subtract(x, a)
print(sub.eval())

sess.close()

[-2. -1.]


## 5. Tensors
- tensor-flow에서 사용하는데이터 구조
- n차원의 배열이나 리스트
- static type, rank(n-dim), shape(nxm)

## 6. Variables
- variable은 파라미터 변수로 표현

In [8]:
state = tf.Variable(0, name = "counter") # 변하는 값
one = tf.constant(1) # 상수 값

# a + 1
new_value = tf.add(state, one)

# a = a + 1
update = tf.assign(state, new_value) 

# init을 이용해 변수 초기화
init_op = tf.global_variables_initializer()

# Session 실행
with tf.Session() as sess:
    sess.run(init_op)
    
    # 시작 state
    print(sess.run(state))
    
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

0
1
2
3


## 7. Fetches
- 복수의 tensor를 받아 계산하는 것
- subgraphy를 생성해 병렬처리를 가능하게 해준다.

In [15]:
x = tf.constant(2)
y = tf.constant(3)

add_op = tf.add(x, y) # 5
mul_op = tf.multiply(x, y) # 6
useless = tf.multiply(x, add_op) # 10
pow_op = tf.pow(add_op, mul_op) # 5 ** 6 = 15625

with tf.Session() as sess:
    x, not_useless = sess.run([pow_op, useless])
    print(x, not_useless)

15625 10


### 8. Feeds
- graph가 실행 할 때 직접 tensor 값을 주는 것 feed 메커니즘
- 차원이 달라도 실행이 가능
- run의 파라미터로 사용

In [10]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = input1 * input2

with tf.Session() as sess:
    print(sess.run([output], feed_dict={input1:[7.], input2:[2.]}))

[array([ 14.], dtype=float32)]
